Install libraries

In [5]:
!pip install langchain langchain-huggingface langchain-community chromadb transformers torch


2. Import Libraries

In [6]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import FastEmbedEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import os, getpass

3. Authenticate Hugging Face

In [7]:
# Prompt for your Hugging Face API key if not already set
if not os.getenv("HUGGINGFACEHUB_API_TOKEN"):
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Enter your Hugging Face API key: ")


4. Load and Split CTSE Lecture Notes

In [8]:
# Load your lecture notes PDF file
loader = PyPDFLoader("../CTSE_Lecture_Notes.pdf")  # Replace with your file name
pages = loader.load_and_split()

# Split into chunks (important for context-aware retrieval)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True,
)
docs = splitter.split_documents(pages)
print(f"Split {len(pages)} pages into {len(docs)} chunks.")

Split 376 pages into 382 chunks.


5. Create Embeddings & Vector Store

In [9]:
persist_directory = "./chroma_langchain_db"

# Use FastEmbed to convert text into vectors
embeddings = FastEmbedEmbeddings()

if os.path.exists(persist_directory):
    # If already exists, load the existing DB
    vector_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
    print("Loaded existing vector store.")

else:
    # Otherwise, create and save
    vector_store = Chroma.from_documents(
        documents=docs,
        embedding=embeddings,
        persist_directory=persist_directory
    )
    vector_store.persist()
    print("Created and saved new vector store.")


C:\Users\Malki\AppData\Local\Temp\ipykernel_19152\3614847898.py:8: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings)


Loaded existing vector store.


6. Set Up Retriever

In [10]:
# Retriever with threshold filtering
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "k": 3,
        "score_threshold": 0.5,
    }
)

7. Initialize LLaMA 3 via Ollama

In [11]:
# Initialize LLaMA 3 via Ollama
llm = ChatOllama(model="tinyllama")

C:\Users\Malki\AppData\Local\Temp\ipykernel_19152\3118759340.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="tinyllama")


8. Build Retrieval QA Chain

In [12]:
# Prompt using chat template
chat_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the provided context to answer the user's question. If you don't know the answer based on the context, say 'I don't know.'"),
    ("human", "Context:\n{context}\n\nQuestion: {question}"),
])


In [13]:
# Memory for multi-turn conversation
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

C:\Users\Malki\AppData\Local\Temp\ipykernel_19152\4007295912.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [14]:
# Build Conversational Retrieval QA Chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": chat_template}
)

9. Define Chatbot Function

In [15]:
def ask(query: str):
    # Ask a question using the qa_chain
    result = qa_chain.invoke({"question": query})

    # Print the answer
    print("\nAnswer:", result["answer"])

    # Print document sources if available
    if "source_documents" in result:
        for doc in result["source_documents"]:
            print("Source:", doc.metadata.get("source", "Unknown"))


10. Batch Querying – Test Multiple Questions

In [16]:
ask("What are the key principles of CTSE?")

No relevant docs were retrieved using the relevance score threshold 0.5



Answer: Answer: Sure! Based on the provided context, here is a possible answer to the question:

Key Principles of CTSE (Computer Technology Specialist Education):
1. Collaboration and Communication: Teamwork and communication are key components in the development and execution of projects.
2. Innovation and Creativity: The ability to innovate and create new ideas and solutions is essential for success in a technological society.
3. Excellence in Technical Skills: Developing technical skills such as problem-solving, critical thinking, and data analysis is crucial for future job opportunities and career advancement.
4. Professionalism: Professionalism is an essential trait that sets individuals apart in their chosen careers. In CTSE, professionals are expected to act with integrity and exhibit good work ethic.
5. Technology Literacy: Knowledge of technology-related concepts and their applications are highly valued in the workplace.
6. Adaptability: Being able to adapt and pivot to chan

In [17]:
# Try sample questions from your lecture notes
questions = [
    "What is software engineering?",
    "Explain the Agile methodology.",
    "What are current trends in AI and ML?",
    "Define DevOps in software development."
]

for q in questions:
    print(f"\n🟡 Question: {q}")
    print(f"🟢 Answer: {chatbot(q)}")


🟡 Question: What is software engineering?


NameError: name 'chatbot' is not defined

In [18]:
ask("Explain the Agile methodology.")


Answer: The key principles of Agile methodology are Collaboration, Innovatons, Creativity, Excellence in Technical Skills, Professionalism, Adaptability, and Customer Service. These principles enable individuals to work together collaboratively, innovate effectively, create new solutions, adapt quickly to changing environments and processes, exhibit professionalism and adaptability, and provide high-quality customer service.
